In [1]:
import pandas as pd
import numpy as np

In [14]:
data = pd.read_csv('data.csv',index_col=0)
data

,title,clickbait_title
0,Украинская ЛГБТ-активистка обвинила ню-фотогра...,"К сожалению, я не могу ничего сказать об этом...."
1,Онлайн-кинотеатр вырезал «расстрел» мирных рос...,**Эксклюзив: почему PREMIER удалил пятую серию...
2,Сын Валерии попал в третью аварию за неделю,«Сын Валерии попал в третью аварию за неделю: ...
3,«Тоттенхэм» прервал 11-матчевую беспроигрышную...,«Тоттенхэм» прервал беспроигрышную серию «Вулв...
4,Развенчан миф о пользе горячего чая при высоко...,«Чай при простуде: сенсационное разоблачение! ...
...,...,...
995,В России научились продавать алкоголь в интернете,"К сожалению, я не могу ничего сказать об этом...."
996,Пышная девушка рассказала о трудностях жизни п...,«Лисица и яблоки» против всего мира: откровенн...
997,Netflix столкнулся с бойкотом из-за комедии пр...,"К сожалению, я не могу ничего сказать об этом...."
998,«Кристал Пэлас» и «Саутгемптон» сыграли вничью...,"«Сенсация в АПЛ: ""Саутгемптон"" сыграл вничью с..."


In [5]:
news_df = pd.read_csv('./data_download/news.csv')
news_df = news_df[['title', 'text', 'topic']]
news_df.head()

,title,text,topic
0,Россиянам дали советы по выбору чая,Россиянам дали советы при выборе чая. Рекоменд...,Россия
1,В Госдуме назвали японское заявление о Курилах...,Спикер Госдумы Вячеслав Володин назвал угрозой...,Россия
2,Украинская ЛГБТ-активистка обвинила ню-фотогра...,Украинская ЛГБТ-активистка Виктория Гуйвик обв...,Культура
3,Полицейские застрелили порезавшего мать буйног...,В Москве полицейские застрелили мужчину при по...,Силовые структуры
4,Беглого президента Боливии решили арестовать,Исполняющая обязанности президента Боливии Жан...,Мир


In [7]:
news_df['topic'].value_counts()

Россия               10996
Мир                   6457
Экономика             4792
Бывший СССР           3798
Спорт                 3563
Наука и техника       2828
Силовые структуры     2714
Интернет и СМИ        2623
Из жизни              2432
Культура              2339
Путешествия           2029
Ценности              1636
Среда обитания        1484
Нацпроекты            1450
69-я параллель         280
Дом                    254
Оружие                   2
Бизнес                   2
Name: topic, dtype: int64

In [9]:
news_filtered_df = news_df[news_df['topic'].isin(['Культура', 'Экономика', 'Спорт', 'Наука и техника', 'Интернет и СМИ'])]
news_filtered_df.to_csv('./data_download/news_filtered.csv')

In [6]:
data = pd.read_csv('data.csv')
kaggle = pd.read_csv('titles_data.csv', sep=';')
clickbait = pd.DataFrame({'title': data['clickbait_title']})
clickbait['is_clickbait'] = 1

not_clickbait = pd.DataFrame({'title': data['title']})
not_clickbait['is_clickbait'] = 0

df = pd.concat([clickbait, not_clickbait])
kaggle.rename(columns={'titles' : 'title', 'target' : 'is_clickbait'}, inplace=True)
df = pd.concat([kaggle, df])

In [7]:
df

,title,is_clickbait
0,Родственник раскрыл настоящую фамилию Пугачёво...,1
1,Предсказания Матроны Московской на 2024-й год:...,1
2,"Пророчество схимонахини Нины об антихристе, ми...",1
3,«Думал об этом»: что Путин сказал о своем прее...,1
4,Путин поручил уведомить россиян об изменениях ...,1
...,...,...
995,В России научились продавать алкоголь в интернете,0
996,Пышная девушка рассказала о трудностях жизни п...,0
997,Netflix столкнулся с бойкотом из-за комедии пр...,0
998,«Кристал Пэлас» и «Саутгемптон» сыграли вничью...,0


In [8]:
# Удалим дубликаты и пропуски
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

# Удалим заголовки, на которые YandexGPT не дал ответы
no_answer = 'К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?'
df = df[~df['title'].str.startswith(no_answer)]

# Удалим ненужную информацию из заголовков
def clean(text):
    text = text.strip()
    # Фраза предлагающая один из вариантов кликбейта
    if text.startswith('Вот один из'):
        parts = [part.strip() for part in text.split(':')[1:]]
        text = ' '.join(parts)
    # Фраза предлагающая несколько вариантов кликбейта
    if text.startswith('Вот несколько'):
        text = text.split(':')[1].strip()
    # Ненужные знаки при генерации текста
    useless = ['«', '»', '**', '*']
    for to_change in useless:
        text = text.replace(to_change, '')
    return text

df['title'] = df['title'].apply(clean)

C:\Users\user\AppData\Local\Temp\ipykernel_16392\3154633456.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].apply(clean)


In [10]:
df.to_csv('cleaned_data.csv')